In [2]:
import pandas as pd
from nbapy import game, shot_chart, player, scoreboard, team
import datetime

In [3]:
def datascrape(s, e):
    start = datetime.datetime.strptime(s, "%d-%m-%Y")
    end = datetime.datetime.strptime(e, "%d-%m-%Y")
    #end = datetime.datetime.today()
    dates = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    df = pd.DataFrame({"date": dates})
    df["date"] = df["date"].astype(str)
    df[['Year', 'Month', "Day"]] = df["date"].str.split('-', 2, expand=True)
    df["Month"] = df["Month"].str.lstrip('0')
    df["Day"] = df["Day"].str.lstrip('0')
    
    #to limit to five
    #df = df.head()
    dates = []
    for i, row in df.iterrows():
        day = {}
        day["year"] = row["Year"]
        day["month"] = row["Month"]
        day["day"] = row["Day"]
        dates.append(day)
    
    init = pd.DataFrame({
    "GAME_DATE_EST_x": [],
    "GAME_ID": [],
    "TEAM_ABBREVIATION_x": [],
    "TEAM_ABBREVIATION_y": [],
    "TEAM_ID_x": [],
    "TEAM_ID_y": [],
    "PTS_x": [],
    "PTS_y": []
    })
    
    for date in dates:
        scores = scoreboard.Scoreboard(month=int(date["month"]), day=int(date["day"]), year=int(date["year"]), league_id='00', offset=0)
        df = scores.line_score()
        if df.empty:
            pass
        else:
            df_1 = df.drop_duplicates(subset=['GAME_ID'], keep='first')
            df_2 = df.drop_duplicates(subset=['GAME_ID'], keep='last')
            merged = df_1.merge(df_2, on=["GAME_ID"])                                                                              #FG_PCT	FT_PCT	FG3_PCT	AST	REB	TOV
            merge = merged[["GAME_DATE_EST_x", "GAME_ID", "TEAM_ABBREVIATION_x", "TEAM_ID_x", "TEAM_ID_y", "TEAM_ABBREVIATION_y", "PTS_x", "PTS_y", "FG_PCT_x", "FG_PCT_y", "FT_PCT_x", "FT_PCT_y", "FG3_PCT_x", "FG3_PCT_y", "AST_x", "AST_y", "REB_x", "REB_y", "TOV_x", "TOV_y"]]
            init = pd.concat([init, merge], sort=True)
    
    return init

In [4]:
games_1 = datascrape("22-10-2019", "01-01-2020")

In [5]:
games_2 = datascrape("02-01-2020", "17-03-2020")

In [6]:
games_3 = datascrape("19-06-2020", "12-10-2020")

In [7]:
concat = pd.concat([games_1, games_2], sort=True)

In [8]:
concat_2 = pd.concat([concat, games_3], sort=True)

In [9]:
concat_2

,AST_x,AST_y,FG3_PCT_x,FG3_PCT_y,FG_PCT_x,FG_PCT_y,FT_PCT_x,FT_PCT_y,GAME_DATE_EST_x,GAME_ID,PTS_x,PTS_y,REB_x,REB_y,TEAM_ABBREVIATION_x,TEAM_ABBREVIATION_y,TEAM_ID_x,TEAM_ID_y,TOV_x,TOV_y
0,30.0,23.0,0.422,0.350,0.422,0.408,0.850,0.842,2019-10-22T00:00:00,0021900001,122.0,130.0,53.0,57.0,NOP,TOR,1.610613e+09,1.610613e+09,19.0,16.0
1,20.0,24.0,0.394,0.355,0.435,0.519,0.714,0.708,2019-10-22T00:00:00,0021900002,102.0,112.0,41.0,45.0,LAL,LAC,1.610613e+09,1.610613e+09,14.0,14.0
0,25.0,28.0,0.300,0.523,0.467,0.511,0.818,0.813,2019-10-23T00:00:00,0021900003,125.0,126.0,49.0,41.0,CHI,CHA,1.610613e+09,1.610613e+09,10.0,19.0
1,26.0,26.0,0.458,0.381,0.526,0.461,0.788,0.833,2019-10-23T00:00:00,0021900004,119.0,110.0,45.0,36.0,DET,IND,1.610613e+09,1.610613e+09,18.0,15.0
2,20.0,24.0,0.265,0.300,0.375,0.430,0.667,0.579,2019-10-23T00:00:00,0021900005,85.0,94.0,54.0,46.0,CLE,ORL,1.610613e+09,1.610613e+09,16.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,29.0,32.0,0.407,0.340,0.507,0.505,0.912,0.588,2020-10-02T00:00:00,0041900402,114.0,124.0,37.0,44.0,MIA,LAL,1.610613e+09,1.610613e+09,9.0,9.0
0,23.0,25.0,0.333,0.353,0.430,0.513,0.759,0.913,2020-10-04T00:00:00,0041900403,104.0,115.0,43.0,37.0,LAL,MIA,1.610613e+09,1.610613e+09,19.0,12.0
0,25.0,18.0,0.359,0.344,0.443,0.427,0.857,0.808,2020-10-06T00:00:00,0041900404,102.0,96.0,42.0,39.0,LAL,MIA,1.610613e+09,1.610613e+09,15.0,11.0
0,26.0,21.0,0.424,0.368,0.458,0.463,0.955,0.857,2020-10-09T00:00:00,0041900405,111.0,108.0,35.0,41.0,MIA,LAL,1.610613e+09,1.610613e+09,13.0,15.0


In [10]:
concat_2["GAME_DATE_EST_x"] = concat_2["GAME_DATE_EST_x"].str.replace("T00:00:00", "")

In [11]:
all_games = concat_2[["GAME_DATE_EST_x",
        "GAME_ID",
        "TEAM_ABBREVIATION_x",
        "TEAM_ABBREVIATION_y",
         "TEAM_ID_x",
         "TEAM_ID_y",
        "PTS_x",
        "PTS_y"]]

In [12]:
all_games = all_games.rename(columns={"GAME_ID": "game_id", "GAME_DATE_EST_x": "date", "TEAM_ABBREVIATION_x": "home", 
                        "TEAM_ABBREVIATION_y": "away", "PTS_x": "home_pts", "PTS_y": "away_pts",
                       "TEAM_ID_x": "home_id", "TEAM_ID_y": "away_id"})
all_games

,date,game_id,home,away,home_id,away_id,home_pts,away_pts
0,2019-10-22,0021900001,NOP,TOR,1.610613e+09,1.610613e+09,122.0,130.0
1,2019-10-22,0021900002,LAL,LAC,1.610613e+09,1.610613e+09,102.0,112.0
0,2019-10-23,0021900003,CHI,CHA,1.610613e+09,1.610613e+09,125.0,126.0
1,2019-10-23,0021900004,DET,IND,1.610613e+09,1.610613e+09,119.0,110.0
2,2019-10-23,0021900005,CLE,ORL,1.610613e+09,1.610613e+09,85.0,94.0
...,...,...,...,...,...,...,...,...
0,2020-10-02,0041900402,MIA,LAL,1.610613e+09,1.610613e+09,114.0,124.0
0,2020-10-04,0041900403,LAL,MIA,1.610613e+09,1.610613e+09,104.0,115.0
0,2020-10-06,0041900404,LAL,MIA,1.610613e+09,1.610613e+09,102.0,96.0
0,2020-10-09,0041900405,MIA,LAL,1.610613e+09,1.610613e+09,111.0,108.0


In [19]:
all_games = all_games.reset_index(drop=True)

In [20]:
dropped = all_games.drop(all_games[all_games.game_id.str[:3] == "001"].index)

In [21]:
dropped

,date,game_id,home,away,home_id,away_id,home_pts,away_pts
0,2019-10-22,0021900001,NOP,TOR,1.610613e+09,1.610613e+09,122.0,130.0
1,2019-10-22,0021900002,LAL,LAC,1.610613e+09,1.610613e+09,102.0,112.0
2,2019-10-23,0021900003,CHI,CHA,1.610613e+09,1.610613e+09,125.0,126.0
3,2019-10-23,0021900004,DET,IND,1.610613e+09,1.610613e+09,119.0,110.0
4,2019-10-23,0021900005,CLE,ORL,1.610613e+09,1.610613e+09,85.0,94.0
...,...,...,...,...,...,...,...,...
1168,2020-10-02,0041900402,MIA,LAL,1.610613e+09,1.610613e+09,114.0,124.0
1169,2020-10-04,0041900403,LAL,MIA,1.610613e+09,1.610613e+09,104.0,115.0
1170,2020-10-06,0041900404,LAL,MIA,1.610613e+09,1.610613e+09,102.0,96.0
1171,2020-10-09,0041900405,MIA,LAL,1.610613e+09,1.610613e+09,111.0,108.0


In [22]:
dropped.to_csv("../Resources/init_games.csv")